In [11]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

In [12]:
# 1. preprocessing data
# 2. build model
# 2.1 encoder
# 2.2 attention
# 2.3 decoder
# 3. evaluation
# 3.1 give data, return result
# 3.2 visualize

path = "../output/demo2.csv"
dir_path = "../output/"
encoding_units = 1024
batch_size = 1
classify = {0: 'ok', 1: 'crosstalked'}

In [13]:
a = pd.read_csv(path, header=None)
b = a.values
# print(type(b))
c = tf.convert_to_tensor(b[:, :-1])
d = tf.convert_to_tensor(b[:, -1:])
print(c.shape[0])
# print(d)
x = tf.fill((3,219), -999)
# x = tf.cast(x, dtype=tf.double)
# y = tf.concat([c, x], axis = 0)
# print(y)

60


In [14]:
# print(os.path.abspath(dir_path))
for filename in os.listdir(dir_path):
    abs_dir_path = os.path.abspath(dir_path)
#     print(filename)
    if filename[-3:] == "csv":
        file_path = abs_dir_path + '/'+ filename
        print(file_path)

/Users/memoforward/Documents/Lab/EMCManagement/code/pythonCode/ML/output/mainboard.csv
/Users/memoforward/Documents/Lab/EMCManagement/code/pythonCode/ML/output/mt18vddt6472ag-265c4.csv
/Users/memoforward/Documents/Lab/EMCManagement/code/pythonCode/ML/output/SampleAD-01.csv
/Users/memoforward/Documents/Lab/EMCManagement/code/pythonCode/ML/output/SampleAD-02.csv
/Users/memoforward/Documents/Lab/EMCManagement/code/pythonCode/ML/output/AD8367_VGA.csv
/Users/memoforward/Documents/Lab/EMCManagement/code/pythonCode/ML/output/demo2.csv


In [15]:
from sklearn.model_selection import train_test_split
def prase_data(file_dir):
    """
    @file_dir: 数据所在文件夹
    """
    maxlen = 0
    embedding_units = 0
    data_list = []
    count = 0
    flag = True
    # 1. 读取文件夹下所有的文件，并获取最大的PCB序列长度
    for filename in os.listdir(dir_path):
        abs_dir_path = os.path.abspath(dir_path)
        if filename[-3:] == "csv":
            file_path = abs_dir_path + '/' + filename
            pd_data = pd.read_csv(file_path, header=None)
            maxlen = max(pd_data.values.shape[0], maxlen)
            if embedding_units == 0:
                embedding_units = pd_data.values.shape[1] - 1 
            elif embedding_units != pd_data.values.shape[1] - 1:
                print("数据有问题，embbedding 维度不一致!")
                return None,None,None
            data_list.append(pd_data)
    print("max_len:", maxlen)
        
    # 2. 读取数据，并获得训练的tensor，返回input_tensor和output_tensor
    for _data in data_list:
        # 2.1 读取数据
        # TODO: 在这里添加数据的预处理,比如归一化什么的
        data = tf.convert_to_tensor(_data.values)
        data = tf.cast(data, dtype=tf.float32)
        # 2.2 拓展PCB序列长度到同一个维度
        padding_len = maxlen - data.shape[0]
        if padding_len > 0:
            padding_value = tf.fill((padding_len, data.shape[1]), 0.1)
            padding_value = tf.cast(padding_value, dtype=tf.float32)
            data = tf.concat([data, padding_value], axis = 0)
            # _input shape: [1, maxlen, embedding_units]
            # _output shape: [1, maxlen, 1]
            _input = tf.expand_dims(data[:, : -1], axis = 0)
            _output = tf.expand_dims(data[:, -1 :], axis = 0)
        if flag is not True:
            input_tensor = tf.concat([input_tensor, _input], axis = 0)
            output_tensor = tf.concat([output_tensor, _output], axis = 0)
        else: 
            input_tensor = _input
            output_tensor = _output
            flag = False
        
        # 用于数据集分割，目前数据集太少了，就不分割了
#     input_train, output_train, input_eval, output_eval = train_test_split(input_tensor, output_tensor, test_size = 0.1)
#     return input_train, output_train, input_eval, output_eval
    
    return input_tensor, output_tensor,embedding_units

In [16]:
batch_size = 1
epochs = 20
def make_dataset(input_tensor, output_tensor, batch_size, epochs, shuffle):
    dataset = tf.data.Dataset.from_tensor_slices((input_tensor, output_tensor))
    if shuffle:
        dataset = dataset.shuffle(30000)
        dataset = dataset.repeat(epochs).batch(batch_size, drop_remainder=True)
    return dataset

In [17]:
input_train, output_train, embedding_units = prase_data(dir_path)
train_dataset = make_dataset(input_train, output_train, batch_size, epochs, True)
print("embedding_units:",embedding_units)

max_len: 1161
embedding_units: 219


In [18]:
for (step, (x,y)) in enumerate(train_dataset.take(1)):
    print(x.shape)
    z = tf.reshape(y[:, 1], (-1, ))
    print(z)
    print(z.shape)
    print(step)
#     print(x)
#     print(y)

(1, 1161, 219)
tf.Tensor([0.], shape=(1,), dtype=float32)
(1,)
0


In [27]:
encoding_units = 256
decoding_units = 256
batch_size = 1
# 判断出的分类
classifies = 2

In [28]:
class Encoder(keras.Model):
    def __init__(self, batch_size, encoding_units):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.encoding_units = encoding_units
        self.gru = keras.layers.GRU(self.encoding_units,
                                    return_sequences=True,
                                    return_state=True,
                                    recurrent_initializer='glorot_uniform')

    def call(self, inputs, hidden):
        x = inputs
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size, self.encoding_units))
    
encoder = Encoder(batch_size, encoding_units)
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(x, sample_hidden)
print("sample_output shape:", sample_output.shape)
print("sample_hidden shape:", sample_hidden.shape)

sample_output shape: (1, 1161, 256)
sample_hidden shape: (1, 256)


In [29]:
class BahdanauAttention(keras.Model):
    def __init__(self, attention_units):
        super(BahdanauAttention, self).__init__()
        self.W1 = keras.layers.Dense(attention_units)
        self.W2 = keras.layers.Dense(attention_units)
        self.V = keras.layers.Dense(1)
        
    def call(self, decoder_hidden, encoder_outputs):
        # decoder_hidden shape: (batch_size, encoding_units)
        # encoder_output shape: (batch_size, length, encoding_units)
        decoder_hidden_with_time_axis = tf.expand_dims(decoder_hidden, 1)
        
        # before V: (batch_size, length, attention_units)
#         print( tf.nn.tanh(
#                 self.W1(encoder_outputs) + self.W2(decoder_hidden_with_time_axis)).shape)
        # after V: (batch_size, length, 1)
        score = self.V(
            tf.nn.tanh(
                self.W1(encoder_outputs) + self.W2(decoder_hidden_with_time_axis)))
        
        # shape: (batch_size, length, 1)
        attention_weights = tf.nn.softmax(score, axis = 1)
        
        # shape: (batch_size, length, encoding_units)
        context_vector = attention_weights * encoder_outputs
        
        # shape: (batch_size, encoding_units)
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        return context_vector, attention_weights

attention_model = BahdanauAttention(10)
attention_results, attention_weights = attention_model(sample_hidden, sample_output)
print("attention_results shape:", attention_results.shape)
print("atteion_weigths shape:", attention_weights.shape)

attention_results shape: (1, 256)
atteion_weigths shape: (1, 1161, 1)


In [30]:
class Decoder(keras.Model):
    def __init__(self, classifies, decoding_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.decoding_units = decoding_units
        self.gru = keras.layers.GRU(self.decoding_units,
                                    return_sequences=True,
                                    return_state=True,
                                    recurrent_initializer='glorot_uniform')
        self.fc = keras.layers.Dense(classifies)
        self.attention = BahdanauAttention(self.decoding_units)

    def call(self, inputs, hidden, encoding_outputs):
        # [b, 1, embedding_units]： single step decoder
        x = tf.cast(inputs, dtype=tf.float32)

        # context_vector: [b, enconding_units]
        context_vector, attention_weights = self.attention(
            hidden, encoding_outputs
        )

        # [b, 1, encoding_units + embedding_units]
        combined_x = tf.concat(
            [tf.expand_dims(context_vector, axis=1), x], axis=-1
        )

        # output shape: [b, 1, decoding_units]
        # state shape: [b, decoding_units]
        output, state = self.gru(combined_x)

        # [b, 1, decoding_units] -> [b, decoding_units]
        output = tf.reshape(output, shape=(-1, output.shape[2]))

        # [b, classifies]
        predictions = self.fc(output)

        return predictions, state, attention_weights
    
decoder = Decoder(classifies, decoding_units, batch_size)
outputs = decoder(tf.random.uniform((batch_size, 1, embedding_units)), sample_hidden, sample_output)

decoder_output, decoder_hidden, decoder_aw = outputs

print("decoder_output shape:", decoder_output.shape)
print("decoder_hidden shape:", decoder_hidden.shape)
print("decoder_attention_weights shape:", decoder_aw.shape)

decoder_output shape: (1, 2)
decoder_hidden shape: (1, 256)
decoder_attention_weights shape: (1, 1161, 1)


In [35]:
optimizer = keras.optimizers.Adam()

# 这里reduction设置成none是为了将sentence中padding的部分做去除操作，先不聚合
loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction = 'none')


def loss_function(real, pred):
    # 以下代码用于去除padding，因为每个PCB板SI结构肯定不一样多
    mask = tf.math.logical_not(tf.math.equal(real, 0.1))
    loss_ = loss_object(real, pred)
#     print(loss_.shape)
#     print(loss_)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

# loss_function(decoder_output, y[:,0])

In [36]:
def train_step(inp, targ, encoding_hidden):
    # inp: [b, len, embedding_units]
    # targ: [b, len, classifies]
    loss = 0
    acc  = 0
    with tf.GradientTape() as tape:
        encoding_outputs, encoding_hidden = encoder(inp, encoding_hidden)
        decoding_hidden = encoding_hidden

        # every SI model's prediction should calculate losses
        for t in range(0, targ.shape[1] - 1):
            # use original inp to predict
            decoding_input = tf.expand_dims(inp[:, t, :], axis=1)
#             print(decoding_input.shape)

            predictions, decoding_hidden, _ = decoder.call(decoding_input, decoding_hidden, encoding_outputs)
            loss += loss_function(tf.reshape(targ[:, t],(-1,)), predictions)

    batch_loss = loss / int(targ.shape[0])
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss

In [33]:
a = input_train
print(a.shape)
print(a[:,1,:].shape)

(6, 1161, 219)
(6, 219)


In [37]:
epochs = 10
steps_per_epoch = len(input_train) // batch_size

for epoch in range(epochs):
    start = time.time()
    
    encoding_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    total_acc = 0
    for (batch, (inp, targ)) in enumerate(
        train_dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, encoding_hidden)
        total_loss += batch_loss
#         if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {: .4f}'.format(epoch + 1, batch, batch_loss.numpy()))
    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
    
    print('Time take for 1 epoch {} sec \n'.format(time.time() - start))

Epoch 1 Batch 0 Loss  26.3959


KeyboardInterrupt: 

In [ ]:
def evaluate(input_data):
    result = []
    # attention_matrix = np.zeros()
    # encoding_hidden = encoder.initialize_hidden_state()
    encoding_hidden = tf.random.normal((1, encoding_units))
    _inputs = tf.expand_dims(input_data, axis=0)
    encoding_outputs, encoding_hidden = encoder(_inputs, encoding_hidden)
    print("encoding_outputs:", encoding_outputs.shape)
    print("encoding_hidden:", encoding_hidden.shape)
    decoding_hidden = encoding_hidden
    decoding_input = tf.zeros((1, 1, input_data.shape[1]))
    print("decoding_input:", decoding_input.shape)
    for t in range(input_data.shape[0]):
        pre, decoding_hidden, attention_weights = decoder.call(decoding_input, decoding_hidden, encoding_outputs)
        # attention_weights = tf.reshape(attention_weights, (-1, ))
        # attention_matrix[t] =  attention_weights.numpy()

        predicted_id = tf.argmax(pre[0]).numpy()
        result.append(classify.get(predicted_id))
        decoding_input = tf.expand_dims(tf.expand_dims(input_data[t], axis=0), axis = 0)
    return result